In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pa
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import keras

#datafile = pa.read_excel('/content/drive/My Drive/Colab Notebooks/melbourne-house-prices.xlsx', sheet_name='ass2_prep_Melbourne_housing_FUL')

datafile = pa.read_csv("/content/drive/My Drive/Colab Notebooks/cleaned_data.csv")

datafile['max_bound_100K'].replace(regex=r'(K$)', inplace=True, value='')
datafile['max_bound_100K'] = datafile['max_bound_100K'].astype(int)

counts = datafile['max_bound_100K'].value_counts().to_dict()

In [0]:
#merge classes so that each has around 1500 elements, though classes whose base size exceed 1500 may go significantly over
sum = 0
previous = 0

for x in sorted(counts):
    sum += counts[x]
    if sum > 1500:
        sum = 0
        previous = 0
    if previous != 0:
        datafile.loc[datafile.max_bound_100K == previous, 'max_bound_100K'] = x
    previous = x

In [0]:
Y = datafile['max_bound_100K']

#remove street and unit address as too granular for our uses
datafileReduced = datafile.copy()
#datafileReduced['address'].replace(regex=r'(^[0-9]{1,}[/][0-9]{1,}[ ])|(^[0-9]{1,}[ ])', inplace=True, value='')
#remove extraneous columns, or columns missing values
#datafileReduced = datafileReduced.drop(["min_bound_100K", "max_bound_100K", "landsize", "building_area", "year_built", "lattitude", "longtitude", "date", "bedrooms", "bathrooms", "car_parks", "address", "price_bands"], axis=1)

datafileReduced = datafileReduced.drop(["max_bound_100K", "lattitude", "longtitude", "price_bands"], axis=1)

In [0]:
datafileDummies = pa.get_dummies(datafileReduced, drop_first=True)
datafileDummies = pa.get_dummies(datafileDummies, columns=["postcode"])
dataset = datafileDummies.values

yDummies = pa.get_dummies(Y, columns=["max_bound_100K"], drop_first=True)
yDataset = yDummies.values

In [0]:
X = dataset

In [0]:
scaledX = preprocessing.MinMaxScaler().fit_transform(X)

trainX, testX, trainY, testY = train_test_split(scaledX, yDataset, test_size=0.3)

In [93]:
model = Sequential([
    Dense(512, activation='sigmoid', input_shape=(954,)),
    Dense(512, activation='sigmoid'),
    Dense(11, activation='softmax'),
])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

hist = model.fit(trainX, trainY, 
           batch_size=64, epochs=10, 
           validation_split=0.2)

Train on 15248 samples, validate on 3812 samples
Epoch 1/10
15248/15248 [==============================] - 6s 398us/step - loss: 2.1433 - acc: 0.1674 - val_loss: 1.8512 - val_acc: 0.2403
Epoch 2/10
15248/15248 [==============================] - 5s 308us/step - loss: 1.6867 - acc: 0.2884 - val_loss: 1.6071 - val_acc: 0.3311
Epoch 3/10
15248/15248 [==============================] - 5s 306us/step - loss: 1.5385 - acc: 0.3469 - val_loss: 1.5512 - val_acc: 0.3437
Epoch 4/10
15248/15248 [==============================] - 5s 310us/step - loss: 1.4880 - acc: 0.3740 - val_loss: 1.5352 - val_acc: 0.3434
Epoch 5/10
15248/15248 [==============================] - 5s 316us/step - loss: 1.4557 - acc: 0.3922 - val_loss: 1.5151 - val_acc: 0.3751
Epoch 6/10
15248/15248 [==============================] - 5s 309us/step - loss: 1.4380 - acc: 0.3999 - val_loss: 1.5236 - val_acc: 0.3646
Epoch 7/10
15248/15248 [==============================] - 5s 315us/step - loss: 1.4264 - acc: 0.4067 - val_loss: 1.5202 - v

In [94]:
model.evaluate(testX, testY)[1]

8169/8169 [==============================] - 1s 128us/step


0.3792385849050566

In [0]:
model.save('256_128_relu_shrink2_9.h5')